# 자연스러운 코드로

- 가변 길이 인수 (순전파)
- 가변 길이 인수 (개선)
- 가변 길이 인수 (역전파)


## 가변 길이 인수 (순전파)

```python
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # Variable 인스턴스로부터 데이터를 꺼낸다.
        ys = self.forward(xs) # forward 메서드에서 구체적인 계산을 수행한다.
        outputs = [Variable(as_array(y)) for y in ys] # 계산된 데이터를 Variable 인스턴스로 다시 감싼다.

        for output in outputs:
            output.set_creator(self) # 원산지 표시를 한다.

        self.inputs = inputs  # 입력 변수를 기억(보관)한다.
        self.outputs = outputs  # 출력 변수를 저장한다.
        return outputs

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()
```

인수와 반환값을 리스트로 변경했다.


### Add 클래스

```python
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)  # 반환값을 튜플로 묶는다.
```


In [6]:
# import numpy as np
# from framework.variable import Variable
# from framework.function import Add
# xs = [Variable(np.array(2)), Variable(np.array(3))]
# f = Add()
# ys = f(xs)
# y = ys[0]
# print(y.data)

## 가변 길이 인수 (개선)

### 함수를 사용하기 쉽게 개선

```python
class Function:
  def __call__(self, *inputs):  # *를 붙여 가변 길이 인수로 변경
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)  # 리스트 대신 언패킹하여 넘긴다.
    if not isinstance(ys, tuple):  # 튜플이 아닌 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]
```


In [7]:
import numpy as np
from framework.function import Add
from framework.variable import Variable
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


In [8]:
from framework.function import add
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## 13 가변 길이 인수 (역전파)

### 13.1 가변 기이 인수에 대응한 Add클래스의 역전파

```python
class Add(Function):
    def forward(self, *xs):
        x0, x1 = xs
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy
```


### 13.2 Variable 클래스의 backward 메서드 수정

```python
def backward(self):
        """재귀에서 반복문으로 변경"""

        if self.grad is None:
            self.grad = np.ones_like(self.data)

        functions = [self.creator]

        while functions:
            f = functions.pop()  # 함수를 가져온다.
            # x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

            # if x.creator is not None:
            #     functions.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.

            gys = [output.grad for output in f.outputs] # 출력변수인 output에서 출력변수의 미분을 가져온다.
            gxs = f.backward(*gys) # 역전파 호출
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 추가 지원
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분을 Variable의 인스턴스 변수 grad에 저장한다.
                x.grad = gx

                if x.creator is not None: # 하나 앞의 함수를 리스트에 추가한다.
                    functions.append(x.creator)
```


### 13.3 Square 클래스 구현

```python
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 수정 전: x = self.input.data
        gx = 2 * x * gy
        return gx
```


In [9]:
from framework.variable import Variable
from framework.function import add, square
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


## 14 같은 변수 반복 사용

현재는 같은 변수를 반복해서 사용할 경우 결과값이 지랄맞게 나온다.


In [10]:
import numpy as np

class Variable:
    def __init__(self, data: np.ndarray):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')
        self.data = data  # 데이터 ndarray
        self.grad = None  # 미분값을 저장하는 변수 ndarray
        self.creator = None  # 변수의 창조자(creator)를 기억하는 변수

    def set_creator(self, func):
        self.creator = func

    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input  # 함수의 입력을 가져온다.
    #         x.grad = f.backward(self.grad)  # 함수의 backward 메서드를 호출한다.
    #         x.backward()  # 하나 앞 변수의 backward 메서드를 호출한다.

    def backward(self):
        """재귀에서 반복문으로 변경"""

        if self.grad is None:
            self.grad = np.ones_like(self.data)

        functions = [self.creator]

        while functions:
            f = functions.pop()  # 함수를 가져온다.
            # x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

            # if x.creator is not None:
            #     functions.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.
            
            gys = [output.grad for output in f.outputs] # 출력변수인 output에서 출력변수의 미분을 가져온다.
            gxs = f.backward(*gys) # 역전파 호출
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 추가 지원
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분을 Variable의 인스턴스 변수 grad에 저장한다.
                x.grad = gx # 같은 변수를 반복 사용할 경우 덮어쓰기 때문에 주의해야 한다.
                # if x.grad is None:
                #     x.grad = gx
                # else:
                #     x.grad = x.grad + gx
                
                if x.creator is not None: # 하나 앞의 함수를 리스트에 추가한다.
                    functions.append(x.creator)


# from framework.variable import Variable
from framework.function import add, square

x = Variable(np.array(3.0))
y = add(x, x)

print('y: ', y.data)

y.backward()
print('x.grad: ', x.grad)

AttributeError: 'Variable' object has no attribute 'generation'

In [ ]:
import numpy as np


class Variable:
    def __init__(self, data: np.ndarray):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')
        self.data = data  # 데이터 ndarray
        self.grad = None  # 미분값을 저장하는 변수 ndarray
        self.creator = None  # 변수의 창조자(creator)를 기억하는 변수

    def set_creator(self, func):
        self.creator = func

    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input  # 함수의 입력을 가져온다.
    #         x.grad = f.backward(self.grad)  # 함수의 backward 메서드를 호출한다.
    #         x.backward()  # 하나 앞 변수의 backward 메서드를 호출한다.

    def backward(self):
        """재귀에서 반복문으로 변경"""

        if self.grad is None:
            self.grad = np.ones_like(self.data)

        functions = [self.creator]

        while functions:
            f = functions.pop()  # 함수를 가져온다.
            # x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

            # if x.creator is not None:
            #     functions.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.
            
            gys = [output.grad for output in f.outputs] # 출력변수인 output에서 출력변수의 미분을 가져온다.
            gxs = f.backward(*gys) # 역전파 호출
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 추가 지원
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분을 Variable의 인스턴스 변수 grad에 저장한다.
                # x.grad = gx # 같은 변수를 반복 사용할 경우 덮어쓰기 때문에 주의해야 한다.
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None: # 하나 앞의 함수를 리스트에 추가한다.
                    functions.append(x.creator)

from framework.function import add, square
x = Variable(np.array(3.0))
y = add(x, x)

print('y: ', y.data)

y.backward()
print('x.grad: ', x.grad)

In [ ]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print('x.grad: ', x.grad)

### 14.3 미분값 재설정


In [ ]:
from framework.function import add, square
from framework.variable import Variable

x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad: ', x.grad)

y = add(add(x, x), x)
y.backward()
print('x.grad: ', x.grad)


In [ ]:
from framework.function import add, square
from framework.variable import Variable

x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad: ', x.grad)
x.cleargrad() # 미분값 초기화
y = add(add(x, x), x)
y.backward()
print('x.grad: ', x.grad)


## 15 복잡한 계산 그래프 (이론)


## 16 복잡한 계산 그래프 (구현)


### 16.1 세대 추가


In [14]:
from framework.variable import Variable
from framework.function import add, square
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()
print(y.data)
print(x.grad)

32.0
64.0
